In [ ]:
# KorBERT: Fine-tuning

In [ ]:
import transformers
transformers.__version__

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [ ]:
# 데이터 불러오기
train_data = pd.read_csv('./data/spam.csv')

train_data['label'].loc[train_data['label']=='ham'] = 0
train_data['label'].loc[train_data['label']=='spam'] = 1

In [ ]:
test_data = pd.read_csv('./data/spam_test_text.csv')
test_label = pd.read_csv('./data/spam_test_label.csv')

test_data.drop('id', axis=1, inplace=True)
test_label.drop('id', axis=1, inplace=True)
test_data = pd.concat([test_data, test_label], axis=1)

test_data['label'].loc[test_data['label']=='ham'] = 0
test_data['label'].loc[test_data['label']=='spam'] = 1

In [ ]:
print('훈련용 리뷰 개수 :', len(train_data)) # 훈련용 리뷰 개수 출력

In [ ]:
print('테스트용 리뷰 개수 :', len(test_data)) # 테스트용 리뷰 개수 출력

In [ ]:
train_data[:5] # 상위 5개 출력

In [ ]:
test_data[:5]

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.drop_duplicates()
train_data = train_data.reset_index(drop=True)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
# test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
# test_data = test_data.drop_duplicates()
# test_data = test_data.reset_index(drop=True)
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print(len(train_data))

In [ ]:
print(len(test_data))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
max_seq_len = 128

In [ ]:
bs = 32
epoch = 2
learning_rate = 5e-5

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['text'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
test_X, test_y = convert_examples_to_features(test_data['text'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
# 최대 길이: 128
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

In [ ]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [ ]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model = TFBertForSequenceClassification("klue/bert-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
hist = model.fit(train_X, train_y, epochs=epoch, batch_size=bs, validation_split=0.2)

In [ ]:
# results = model.evaluate(test_X, test_y, batch_size=1024)
# print("test loss, test acc: ", results)

In [ ]:
from sklearn.metrics import *

pred = model.predict(test_X)

In [ ]:
pred = model.prpred = np.where(pred>0.9, 1, 0)
pred = pred.reshape(-1).astype(int)

acc = accuracy_score(test_y, pred)
f1_log = f1_score(test_y, pred)

acc, f1_log